# Librerías

In [1]:
#!pip install pysqlite3
#!pip install scikit-lear
!pip install numpy==1.23.5

In [2]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
from collections import Counter
from google.colab import drive
from google.colab import files
import os
import sys
from ipywidgets import interact
from sklearn import neighbors
import joblib
#Librerias estadísticas y de pruebas
import scipy.stats as stats
from scipy.stats import gaussian_kde
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler

import datetime




# Conectar con google drive


In [3]:
drive.flush_and_unmount()

In [4]:
#drive.flush_and_unmount()  #Linea en caso de tener que desconectar el drive por algún tipo de falla
drive.mount('/content/drive') #Linea para conectar al drive

Mounted at /content/drive


In [5]:
path="/content/drive/MyDrive/analitica 3/sistemas_recomendacion" ### ruta del repositorio en drive
os.chdir(path) ### volver la carpeta del repositorio directorio de trabajo
sys.path.append(path) ### agregarla al path, poder leer archivos de funciones propios como paquetes

In [6]:
import a_funciones as fn


# Base de datos

In [7]:
conn = sql.connect('/content/drive/MyDrive/analitica 3/sistemas_recomendacion/data/db_movies3')
cur = conn.cursor()

In [8]:
# Creación de cursor para  ejecutar consultas en la base de datos
# Visualizar las tablas contenidas en la base de datos
cur.execute("SELECT name FROM sqlite_master where type='table'")
cur.fetchall()

[('movies_final',), ('ratings_final',), ('df_final',), ('df_terminado',)]

#Modelos

##Sistemas de recomendación basados en popularidad

In [9]:
#Peliculas mejor calificadas por más de 120 usuarios (TOP 50)
pd.read_sql("""select title, rating as rat,
                count (*) as count
                from df_terminado
                group by title
                having count > 120
                order by rat desc
                limit 50""",conn)

,title,rat,count
0,X-Men (2000),5.0,133
1,"Usual Suspects, The (1995)",5.0,204
2,Titanic (1997),5.0,140
3,"Terminator, The (1984)",5.0,131
4,Star Wars: Episode VI - Return of the Jedi (1983),5.0,196
5,Star Wars: Episode V - The Empire Strikes Back...,5.0,211
6,Star Wars: Episode IV - A New Hope (1977),5.0,251
7,Speed (1994),5.0,171
8,Seven (a.k.a. Se7en) (1995),5.0,203
9,Schindler's List (1993),5.0,220


In [10]:
#Peliculas con mejor promedio de calificación (TOP 20)
pd.read_sql("""select title, avg(rating) as avg_rat,
                count (*) as count
                from df_terminado
                group by title
                order by avg_rat desc
                limit 20""",conn)

,title,avg_rat,count
0,Secrets & Lies (1996),4.590909,11
1,Guess Who's Coming to Dinner (1967),4.545455,11
2,Paths of Glory (1957),4.541667,12
3,"Streetcar Named Desire, A (1951)",4.475000,20
4,"Celebration, The (Festen) (1998)",4.458333,12
5,"Shawshank Redemption, The (1994)",4.445289,317
6,Ran (1985),4.433333,15
7,Yojimbo (1961),4.429102,13
8,His Girl Friday (1940),4.392857,14
9,Three Colors: Red (Trois couleurs: Rouge) (1994),4.379895,16


In [11]:
#Peliculas mejor calificadas en promedio por año de publicación
best_movyear=pd.read_sql("""select year_movies,title, avg(rating) as avg_rat,
                count(*) as count
                from df_final
                group by year_movies, title
                order by year_movies desc, avg_rat desc""",conn)
best_movyear2= best_movyear.sort_values(['year_movies', 'avg_rat'], ascending=[False, False]) \
                      .groupby('year_movies', as_index=False).first()
best_movyear2

,year_movies,title,avg_rat,count
0,1940.0,His Girl Friday (1940),4.392857,14
1,1941.0,"Maltese Falcon, The (1941)",4.170455,44
2,1942.0,Casablanca (1942),4.240000,100
3,1944.0,Double Indemnity (1944),4.323529,17
4,1946.0,Notorious (1946),4.250000,20
...,...,...,...,...
72,2014.0,Nightcrawler (2014),4.166667,18
73,2015.0,Spotlight (2015),4.157895,19
74,2016.0,Arrival (2016),3.980769,26
75,2017.0,Logan (2017),4.280000,25


##Sistema de recomendación basado en contenido-Manual

In [12]:
df_final=pd.read_sql("""select * from df_final""",conn)
df_terminado=pd.read_sql("""select * from df_terminado""",conn)

In [13]:
df_modelos=df_terminado.drop(columns=['title','movie_id','user_id'])
joblib.dump(df_modelos,"/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_modelos.joblib")

['/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_modelos.joblib']

In [14]:
sc=MinMaxScaler()
df_final[["year_ratings",'month','day','year_movies']]=sc.fit_transform(df_final[["year_ratings",'month','day','year_movies']])
joblib.dump(df_final,"/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_final.joblib")

['/content/drive/MyDrive/analitica 3/sistemas_recomendacion/salidas/df_final.joblib']

In [15]:
movie='Avengers: Infinity War - Part I (2018)'
ind_movie=df_final[df_final['title']==movie].index.values.astype(int)[0] ### indice del libro en dataframe escalado y dummificado
row_sel_movie =df_modelos.iloc[ind_movie,:] ### seleccionar fila de caracteristicas de libro seleccionados
similar_movies=df_modelos.corrwith(row_sel_movie,axis=1) ##calcular correlación de catalogo vs libro seleccionado
similar_movies=similar_movies.sort_values(ascending=False) ### ordener libros de mayor a menor correlación
top_similar_movies=similar_movies.to_frame(name="correlación").iloc[0:11,] ### el 11 es número de libros recomendados
top_similar_movies['title']=df_final["title"] ### agregaro los nombres (como tiene mismo indice no se debe cruzar)

In [16]:
top_similar_movies

,correlación,title
3306,1.000000,Avengers: Infinity War - Part I (2018)
73603,0.999913,Thor: Ragnarok (2017)
73601,0.999810,Black Panther (2017)
12515,0.999537,Thor: Ragnarok (2017)
12514,0.999535,Avengers: Infinity War - Part I (2018)
47209,0.999258,Thor: Ragnarok (2017)
72003,0.999229,Guardians of the Galaxy 2 (2017)
72000,0.999229,Black Panther (2017)
72005,0.999226,Doctor Strange (2016)
71994,0.999202,Guardians of the Galaxy (2014)


In [17]:
def sist_recomendacion(movie=list(df_final['title'])):
    ind_movie=df_final[df_final['title']==movie].index.values.astype(int)[0] ### indice del libro en dataframe escalado y dummificado
    similar_movies=df_modelos.corrwith(df_modelos,axis=1) ##calcular correlación de catalogo vs libro seleccionado
    similar_movies=similar_movies.sort_values(ascending=False) ### ordener libros de mayor a menor correlación
    top_similar_movies=similar_movies.to_frame(name="correlación").iloc[0:11,] ### el 11 es número de libros recomendados
    top_similar_movies['title']=df_final["title"] ### agregaro los nombres (como tiene mismo indice no se debe cruzar)

    return top_similar_movies

print(interact(sist_recomendacion))

interactive(children=(Dropdown(description='movie', options=('Toy Story (1995)', 'Grumpier Old Men (1995)', 'H…

<function sist_recomendacion at 0x78501e23a8e0>


##Sistema de recomendación basado en contenido KNN

In [18]:
df_modelos = df_modelos.reset_index(drop=True)
df_final = df_final.reset_index(drop=True)

In [19]:
model=neighbors.NearestNeighbors(n_neighbors=10,metric='euclidean')
model.fit(df_modelos)
dist,idlist=model.kneighbors(df_modelos)

In [20]:
distancias=pd.DataFrame(dist)
id_list=pd.DataFrame(idlist)

In [21]:
movie_list_name = []
movie_name='Avengers: Infinity War - Part I (2018)'
movie_id = df_final[df_final['title'] == movie_name].index ### extraer el indice del libro
movie_id = movie_id[0] ## si encuentra varios solo guarde uno

for newid in idlist[movie_id]:
    title = df_final.loc[newid].title
    if title not in movie_list_name:
        movie_list_name.append(title)

movie_list_name

['Avengers: Infinity War - Part I (2018)',
 'Thor: Ragnarok (2017)',
 'Black Panther (2017)',
 'Guardians of the Galaxy 2 (2017)',
 'Doctor Strange (2016)',
 'Guardians of the Galaxy (2014)']

In [22]:
def movie_recomender(movie_name=list(df_final['title'].value_counts().index)):
    movie_list_name = []
    movie_id = df_final[df_final['title'] == movie_name].index ### extraer el indice del libro
    movie_id=movie_id[0] ## si encuentra varios solo guarde uno

    for newid in idlist[movie_id]:
        title = df_final.loc[newid].title
        if title not in movie_list_name:
            movie_list_name.append(title)
    return movie_list_name

print(interact(movie_recomender))

interactive(children=(Dropdown(description='movie_name', options=('Forrest Gump (1994)', 'Shawshank Redemption…

<function movie_recomender at 0x78501ce065c0>
